# Class 8 - EEB125H1

- Bootstrap

- Introduction to regression

# Bootstrap 

A data scientist is using the data in a random sample to estimate an unknown statistical parameter. She uses the sample to calculate the value of a statistic that she will use as her estimate.

Once she has calculated the observed value of her statistic, she could just present it as her estimate and go on her merry way. But she’s a data scientist. She knows that her random sample is just one of numerous possible random samples, and thus her estimate is just one of numerous plausible estimates.

By how much could those estimates vary? To answer this, it appears as though she needs to draw another sample from the population, and compute a new estimate based on the new sample. But she doesn’t have the resources to go back to the population and draw another sample.

It looks as though the data scientist is stuck.

Fortunately, a brilliant idea called the bootstrap can help her out. Since it is not feasible to generate new samples from the population, the bootstrap generates new random samples by a method called resampling: the new samples are drawn at random from the original sample.

- We observed before that a large random sample from a population resembles the population from which it was drawn.

- Consider gestation in days from the pantheria dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

pantheria = pd.read_csv('pantheria.txt',sep="\t")

print(pantheria.shape)

In [ ]:
pantheria['9-1_GestationLen_d'].plot.hist(color = 'grey', edgecolor = 'black');

In [ ]:
pantheria['9-1_GestationLen_d'].median()

- Draw a random sample with replacement of the same size (number of rows) as the original sample.

- This opens the possibility for the new sample to be different from the original sample.

In [ ]:
gest_bsample = pantheria['9-1_GestationLen_d'].sample(frac = 1, replace=True)

gest_bsample.plot.hist(color = 'grey', edgecolor = 'black');

In [ ]:
gest_bsample.median()

## Why is this a good idea?

- By the law of averages, the distribution of the original sample is likely to resemble the population.  

- The distributions of all the “resamples” are likely to resemble the original sample. 

- So the distributions of all the resamples are likely to resemble the population as well.

## Resampled median

- We generated one resampled or *bootstrapped* median above.

- By resampling many times we can compute the empirical distribution of the median gestation time.

In [ ]:
def one_bs_median():
    gest_med_bsample = pantheria['9-1_GestationLen_d'].sample(frac = 1, replace=True).median()
    return gest_med_bsample

In [ ]:
one_bs_median()

- Now let's compute many bootstrap medians by writing a `for` loop.

In [ ]:
bootstrap_medians = []  # empty list to collect medians

for _ in range(5000):
    bootstrap_medians.append(one_bs_median())

In [ ]:
plt.hist(bootstrap_medians, color = 'grey', edgecolor = 'black');

## Boostrap confidence intervals

- We can use the bootstrap distribution to construct a range of values such that 95% of the random samples will contain the true median.

- The range of values is called a confidence interval.

- A 95% confidence interval for the median can be constructed finding the 2.5% percentile and the 97.5% percentile.  

- The reason for choosing 2.5% and 97.5% is that 0.05/2 = 0.025, and 1 - 0.025 = 0.975.   

- We can do this using the `percentile` function in `numpy`.

In [ ]:
np.percentile(bootstrap_medians, 2.5)

In [ ]:
np.percentile(bootstrap_medians, 97.5)

A 95% bootstrap confidence interval for median is 62.99 days to 78.07 days.

## Bootstrap confidence intervals for other statistics

### Do Primate carnivores have different body sizes compared to Carnivora carnivores?

- Let's create a dataset (`DataFrame`) with primates and carnivora body mass measurements. 

In [ ]:
cols = ['MSW05_Order', '5-1_AdultBodyMass_g']

groups = (pantheria['MSW05_Order'] == 'Primates') | (pantheria['MSW05_Order'] == 'Carnivora')

primcarn_df = pantheria.loc[groups, cols].dropna()

primcarn_df.groupby('MSW05_Order').describe()

In [ ]:
primcarn_df.hist(by = 'MSW05_Order', bins = 10, edgecolor = 'black', color = 'grey');

Create a function to create one bootstrap sample of the median difference.

In [ ]:
# primate rows
primes = primcarn_df['MSW05_Order'] == 'Primates'

# carnivora rows
carns = pantheria['MSW05_Order'] == 'Carnivora'


def bs_one_median_diff():
    # sample body mass with replacement
    bssample = primcarn_df.sample(frac = 1, replace =  True)
    
    # compute difference in medians on bootstrap sample
    bs_med_diff = (bssample.loc[carns, '5-1_AdultBodyMass_g'].median() - 
                   bssample.loc[primes, '5-1_AdultBodyMass_g'].median())
    return bs_med_diff

In [ ]:
bs_one_median_diff()

Now, compute bootstrap distribution for a `for` loop.

In [ ]:
bootst_med_diffs = []

for _ in range(5000):
    bootst_med_diffs.append(bs_one_median_diff())

In [ ]:
plt.hist(bootst_med_diffs, color = 'grey', edgecolor = 'black');

A 95% confidence interval for the median difference is:

In [ ]:
np.percentile(bootst_med_diffs, 2.5), np.percentile(bootst_med_diffs, 97.5) 

The confidence interval contains 0, therefore there is very weak evidence that the medians are different.

## In-class exercise

Modify the code above to compute a 95% bootstrap confidence interval for the difference in means between Carnivora and Primates.  Does the confidence interval contain 0?

# Linear Regression

## Basic idea

- Linear regression is a useful technique for creating models to explain relationships between variables. 

- For use in multiple regression variables the dependent variable must be numeric, and have meaningful numeric values. 

- The independent variables can be interval or categorical variables.

## Example 1: perfect linear relationship between dependent and independent variables

### Data

In [ ]:
import pandas as pd
import numpy as np

data = {'depvar' : np.arange(start = 0, stop = 8, step =1),
       'indvar' : np.arange(start = 0, stop = 8, step =1)}

df = pd.DataFrame(data)

df

### Plot the data

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x = df['indvar'], y = df['depvar'])
plt.xlabel('indvar')
plt.ylabel('depvar');

- The scatter plot shows each pair of points (1, 1), (2, 2), etc. 

- `'indvar'` is sometimes called a predictor variable or covariate, and `'depvar'` is called the dependent variable.

- The dependent variable is predicted exactly by the independent variable.

### Compute the regression line

In [ ]:
from statsmodels.formula.api import ols

regmod = ols('depvar ~ indvar', data = df) # setup the model

- The code above uses the `ols` function from `statsmodels.formula.api` 

- The syntax in the function `ols` function `'depvar ~ indvar'` is a special syntax for describing statistical models.  

- The column name to the **left** of `~` specifies the dependent variable.

- The column name to the **right** of `~` specifies the independent variable(s).

In [ ]:
regmod_fit = regmod.fit() # estimate/fit the model 

After the model is setup then the `fit` function can be applied to the model.  This function computes the equation of the regression line.

In [ ]:
regmod_fit.params # get parameter estimates

- The estimates of the **y-intercept** and **slope** are labelled `Intercept` (1.11e-16 approx 0)  and `indvar` (1.00e+00 approx 1).

- This means the regression equation is: $$\texttt{depvar} = 0 + 1 \times \texttt{indvar}$$

## Example 2: another perfect linear relationship

In [ ]:
data = {'depvar' : np.arange(start = 0, stop = 8, step =1) + 2,
       'indvar' : np.arange(start = 0, stop = 8, step =1)}

df = pd.DataFrame(data)

df

In [ ]:
plt.scatter(x = df['indvar'], y = df['depvar'])
plt.xlabel('indvar')
plt.ylabel('depvar');

In [ ]:
regmod = ols('depvar ~ indvar', data = df) # setup the model

regmod_fit = regmod.fit() # estimate/fit the model 

regmod_fit.params # get parameter estimates

- The scatter plot is similar to Example 1 except that the values of the dependent variable has been increased by 2.

- The equation of the regression line for this data is:  $$\texttt{depvar} = 2 + 1 \times \texttt{indvar} $$

### Interpretation of y-intercept

- When the independent variable is 0 the dependent variable is 2.  This is the meaning of the **y-intercept** value of 2.

### Interpretation of slope

- For a one-unit change in the independent variable the dependent variable changes by 1 unit.

## Example 3: close to linear

- Examples 1 and 2 were perfect linear relationships.

- In this example we examine what happens if the relationship between the dependent and independent variables is almost linear.

In [ ]:
np.random.seed(11) #set the seed so that it's reproducible

data = {'depvar' : np.arange(start = 0, stop = 8, step =1) + 2,
        'indvar' : np.arange(start = 0, stop = 8, step =1) + np.random.uniform(low = 0, high = 2, size = 8)}

df = pd.DataFrame(data)

df

In [ ]:
plt.scatter(x = df['indvar'], y = df['depvar'])
plt.xlabel('indvar')
plt.ylabel('depvar');

In [ ]:
regmod = ols('depvar ~ indvar', data = df) # setup the model

regmod_fit = regmod.fit() # estimate/fit the model 

regmod_fit.params # get parameter estimates

So, now the relationship isn't perfectly linear, but close.  The equation of this regression line is:

$$\texttt{depvar} = 1.63 + 0.92 \times \texttt{indvar}$$

In [ ]:
import seaborn as sns

sns.regplot(x = 'indvar', y = 'depvar', data = df, ci = None);

The `regplot` function in the `seaborn` library will produce a scatter plot with the regression line.

The parameters of `regplot`

- `x` is the independent variable.
- `y` is the dependent variable.
- `ci = None` specifies no confidence interval for the regression line 

### Statistical summary of linear regression

In [ ]:
import warnings
warnings.filterwarnings('ignore') # turn off warnings

regsum = regmod_fit.summary()
regsum.tables[1]

- A statistical summary of the regression model is given above using the `summary` function in `statsmodels`.

- There are three summary tables, but we will only be interested in the second summary table---`regsum.tables[1]` (in this course).

In [ ]:
regsum.tables[1]

What does the statistical summary represent?

- the column labelled `coef` are same values as `regmod_fit.params`.  In this case the average increase `depvar` for a one unit change in `indvar` is 0.9229.

- the `std err` column represents the standard deviation of the intercept and slope (we won't discuss in this course).

- the `t` column represents the value of the t-statistic (we won't discuss in this course).

- the column `P>|t|` represents the p-value corresponding to the null hypothesis that the intercept or slope are equal to 0.  If the value is small then this is evidence against the null hypothesis.  But if the p-value is large then there is no evidence against the null hypothesis.  

- Currently, how small a p-value should be to provide evidence against a null hypothesis is being debated.  Although, the traditional threshold is 0.05 (i.e., a p-value less than or equal to 0.05 is supposed to indicate evidence against the null hypothesis).

- In the example above the p-value for the slope is 0.  This indicates strong evidence that the *true* value of the slope is non-zero.

- The columns `[0.025   0.975]` form a plausible range of values for the y-intercept and slope (formally known as a 95% confidence interval). In this case the plausible values for the slope are 0.745 and 1.101.  If the plausible range includes 0 then this is usually interpreted as not providing evidence against the null hypothesis.  

## Predicted values and residuals

If the values of the independent variable are plugged into the regression equation then we obtain the fitted values.

$$\texttt{depvar} = 1.63 + 0.92 \times \texttt{indvar}$$

In [ ]:
df

The fitted value for the first row of `df` is:

In [ ]:
1.6254 + 0.9229*0.360539

- If the linear regression model is used on an independent variable that is not in the data set used to build the model then it's often referred to as a predicted value.

To extract the fitted values from a regression model use the `fittedvalues` function in `statsmodels`.

In [ ]:
regmod_fit.fittedvalues

The residual is the dependent variable minus the fitted value.  So, for the first row the residual is:

In [ ]:
2 - 1.9581414431

To extract the residuals from a regression model use the `resid` function in `statsmodels`.

In [ ]:
regmod_fit.resid

- If the linear regression model fits the data well then the residuals should be close to 0.

- One indication that the linear regression model fits well is to examine the **diagnostic** scatter plot of residuals versus fitted values.

- A plot that looks like a random scatter of points around 0 indicates that linear regression is an appropriate model.

- Since there are only 8 points it's hard to spot a pattern.

In [ ]:
plt.scatter(regmod_fit.fittedvalues, regmod_fit.resid)
plt.axhline(y = 0)
plt.xlabel('fitted values')
plt.ylabel('residuals')

## Accuracy of linear regression

- There are several measures of accuracy for linear regression.

- Popular measures are R-squared and root-mean squared error.

- R-squared can be calculated from a fitted model regression model using the `rsquared` function in `statsmodels`.

- R-squared is always between 0 and 1.  
     + R-squared of 0 indicates a poor fit
     + R-squared of 1 indicates a perfect fit

In [ ]:
regmod_fit.rsquared

## Example 4: regression with two or more independent variables

It's possible to include more than one independent variable in a regression model.  

In [ ]:
np.random.seed(11) #set the seed so that it's reproducible

data = {'depvar' : np.arange(start = 0, stop = 8, step =1) + 2,
        'indvar1' : np.arange(start = 0, stop = 8, step =1) + np.random.uniform(low = 0, high = 2, size = 8),
        'indvar2' : np.arange(start = 0, stop = 8, step =1) + np.random.normal(loc = 0, scale = 1, size = 8)}

df = pd.DataFrame(data)

df

To include more than one independent variable in a regression model add it to the right side of syntax for describing statistical models.

`'depvar ~ indvar1 + indvar2'`

In [ ]:
regmod = ols('depvar ~ indvar1 + indvar2', data = df) # setup the model

Now `fit` `regmod`.

In [ ]:
regmod_fit = regmod.fit() # estimate/fit the model 

Extract the regression parameter estimates using `params`.

In [ ]:
regmod_fit.params # get parameter estimates

The estimated linear (multiple) linear regression model is:

$$\texttt{depvar} = 1.892482 + 0.549341 \times \texttt{indvar1} + 0.385197 \times \texttt{indvar2}$$

- The y-intercept value (1.892482) is the value of the dependent variable when `indvar1` and `indvar2` are both 0.

-  If `indvar1` changes by one-unit, `depvar` changes (on average) by 0.549341.

-  If `indvar2` changes by one-unit, `depvar` changes (on average) by 0.385197. 

### Statistical summary of a multiple regression model

In [ ]:
multregsummary = regmod_fit.summary()

multregsummary.tables[1]

The statistical summary indicates:

- the p-value for `indvar1` is 0.003 and the range of plausible values for the slope is 0.288 to 0.810.

- the p-value for `indvar2` is 0.010 and the range of plausible values for the slope is 0.137 to 0.634.  

- Since, both p-values are small there is evidence that slope is different from 0, and the range of plausible values does not include 0.

### Accuracy of multiple regression model

In [ ]:
plt.scatter(regmod_fit.fittedvalues, regmod_fit.resid)
plt.axhline(y = 0)
plt.xlabel('fitted values')
plt.ylabel('residuals')

In [ ]:
regmod_fit.rsquared

## A regression model of body mass on longevity

### Data

- pantheria data contains data on longevity and body mass.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

pantheria = pd.read_csv('pantheria.txt',sep="\t")
pantheria.head()

We will be interested in the following columns.

In [ ]:
cols = ['5-1_AdultBodyMass_g', '17-1_MaxLongevity_m', '9-1_GestationLen_d', 'MSW05_Binomial']

panthdat_trans = pantheria[cols]

colnames = {cols[0] : 'bodymass',
            cols[1] : 'longevity',
            cols[2] : 'gestation',
            cols[3] : 'name'}

panthdat_trans = panthdat_trans.copy()

panthdat_trans.rename(columns = colnames, inplace = True)

panthdat_trans.head()

Let's look at the distribution of `'bodymass'`, `'longevity'`, and '`gestation`'.

In [ ]:
panthdat_trans.hist(column = ['bodymass', 'longevity', 'gestation'], bins = 4, color = 'grey', edgecolor = 'black', grid = False);

In [ ]:
plt.scatter(panthdat_trans['bodymass'], panthdat_trans['longevity'], color = 'blue', label = 'bodymass');

plt.scatter(panthdat_trans['gestation'], panthdat_trans['longevity'], color = 'red', label = 'gestation');

plt.legend();

### Transform the data

- This is a bit hard to interpret.  

- One way to deal with data that is clumped together is to trandform the values using $\log_{e}(x)$. 

- The difference between 0.2 and 0.1 is 0.1, but the difference between $\log_{e}(0.2)$ and $\log_{e}(0.1)$ is 0.69. 

In [ ]:
x = 0.2 - 0.1

logx = np.log(0.2) - np.log(0.1)

print('0.2 - 0.1 =', x,'\n \n','log(0.2) - log(0.1)=',logx.round(1))

We can compute $log(x)$ of each `'bodymass'` and `'longevity'` using the `pandas` function `apply` with the `numpy` function `log`.

In [ ]:
panthdat_trans[['bodymass', 'longevity', 'gestation']] = (panthdat_trans[['bodymass', 'longevity', 'gestation']].
                                                          apply(np.log))

It will also be helpful to drop missing values.  This can be done using the `pandas` function `dropna` with the parameter `inplace = True`, so that it modifies the existing `DataFrame`.

In [ ]:
panthdat_trans.dropna(inplace = True)

print(panthdat_trans.isna().sum()) # check for missing values

panthdat_trans.head()

### Plot the data

In [ ]:
sns.pairplot(panthdat_trans);

## A regression model of length of gestation on longevity

### Fit the regression model

In [ ]:
reg_longbmass = ols('longevity ~ bodymass', data = panthdat_trans) # setup the model

reg_longbmass_fit = reg_longbmass.fit() # estimate/fit the model 

## Statistical summary of the regression model

In [ ]:
reg_longbmass_summ = reg_longbmass_fit.summary()

reg_longbmass_summ.tables[1]

- The regression equation is:

$$\texttt{longevity} = 3.4256 + 0.2012\times \texttt{bodymass}$$

- The **slope** indicates that for a 1 gram **increase** in body mass longevity increases by 0.20 months.

- The **y-intercept** indicates that when body mass is 0 then longevity is 3.4 months.

- Are there any species with 0 gram body mass?

- The **y-intercept** in regression models often doesn't have a sensible interpretation, but is often mathematically important for regression to work well.

### Accuracy of regression model

In [ ]:
reg_longbmass_fit.rsquared

In [ ]:
plt.scatter(x = reg_longbmass_fit.fittedvalues , y = reg_longbmass_fit.resid)
plt.axhline(y = 0)
plt.xlabel('fitted values')
plt.ylabel('residuals')

## Training and testing a model

- Ideally we could assess the accuracy of this linear regression model on a new set of data (i.e., longevity in species from another study). 

- But, we don't have this data.

- One trick that data scientists use to fit the model on part of the data (training data) and use the remaining part (testing data) to test the accuracy.

- If we only have one data set then the training data can be obtained by randomly selecting rows from the data to fit the regression model, then the remaining rows can be used to test the accuracy of the model.

### Splitting a `pandas` `DataFrame` by `index`

In [ ]:
data = {'a' : [1, 2, 3, 4, 5]}

df = pd.DataFrame(data)

df.index

A Boolean array that is `True` if the index is `0` or `1` and `False` otherwise. 

In [ ]:
df.index.isin([0,1])

To create a Boolean series that is `False` if the index is `0` or `1` and `True` otherwise we can negate `df.index.isin([0,1])` this Boolean series using the `~` operator `~df.index.isin([0,1])`

In [ ]:
~df.index.isin([0,1])

In [ ]:
df[~df.index.isin([0,1])]

### Creating training and test data sets from a single dataset

### Step 1: 

- split the data into a training set with 75% of the rows.

- use the remaining 25% of the data for testing. 

In [ ]:
np.random.seed(11) # for reproducibility

# randomly select 75% of neighbourhoods
reg_df_train = panthdat_trans.sample(frac = 0.75, replace = False) 

# get index of training data
train_index = reg_df_train.index

- Exclude indicies from `reg_df_train` using `pandas` `isin` function. 

In [ ]:
# exclude rows in training to define test data
reg_df_test = panthdat_trans[~panthdat_trans.index.isin(train_index)]

print(panthdat_trans.shape)

print(reg_df_train.shape)

print(reg_df_test.shape)

### Step 2: Fit the regression model on the training data

In [ ]:
reg_train = ols('longevity ~ bodymass', data = reg_df_train) # setup the model

reg_train_fit = reg_train.fit() # estimate/fit the model 

### Step 3: Compute predicted (fitted) values using training data 

In [ ]:
# use the model fit on the training data to predict longevity rates
# from the test set. 

predvalues_train = reg_train_fit.predict(reg_df_train['bodymass'])

### Step 4: Evaluate accuracy using root mean-squared error on training data

- Another measure of accuracy of regression models.

- Compares observed values of the dependent variable with the predicted values.

- It can be computed using the `rmse` function from `statsmodels`.

 Compute root mean-squared error for the training data.

In [ ]:
from statsmodels.tools.eval_measures import rmse

# compute the accuracy of these predictions

rmse(predvalues_train, reg_df_train['bodymass'])

- The observed longevity deviate from the predicted longevity for each species by 4 months in the training set.

- Is this an acceptable prediction error?

- Let's examine the accuracy of the linear regression model on the test set

- First compute the predicted values using the training data.

### Step 5: Evaluate the accuracy of regression model on the test data using root mean-squared error

- Compute predictions using test data

In [ ]:
predvalues_test = reg_train_fit.predict(reg_df_test['bodymass'])

- Now compute the root mean-squared error for the test data using the model fit on the training data.

In [ ]:
rmse(predvalues_test, reg_df_test['bodymass'])

## Conclusion of data analysis using linear regression model

- The regression model appears to provide an accurate fit the data, although both the scatter plot and plot of residuals indicate that there is a lot of variation in species with lower body masses.

- As the body mass increase longevity increases. 

## Are there other factors in our data that might effect longevity?

In [ ]:
sns.pairplot(panthdat_trans);

- Let's consider gestation.

- Fit the regression model on all the data and produce a statistical summary.

In [ ]:
reg_longbmassgest = ols('longevity ~ gestation + bodymass', data = panthdat_trans) # setup the model

reg_longbmassgest_fit = reg_longbmassgest.fit() # estimate/fit the model 

reg_longbmassgest_summ = reg_longbmassgest_fit.summary()

reg_longbmassgest_summ.tables[1]

- We can see from the scatter plot that `gestation` and `bodymass` have a linear relationship.  When two variables have a linear relationship multiple linear regression models can sometimes produce strange results.

- Although, in this case they seem reasonable.

- It's important that independent variables in a multiple regression model are unrelated.